In [2]:
import os
os.chdir("..")
print(os.getcwd())

/data/akhlak/PromptDataExtraction


In [13]:
import warnings
warnings.simplefilter('ignore')

In [3]:
import sett
import pylogg

sett.load_settings()
pylogg.setLevel(pylogg.DEBUG)

Load OK: settings.yaml


In [11]:
import json
from backend.nlp import ner

def BERT():
    """ Load the MaterialsBERT model and return the model handle. """
    bertModel = ner.MaterialsBERT(sett.NerModel.model)
    bertModel.init_local_model(device=0)
    return bertModel

def Tg_dataset():
    """ Return a list of (doi, text) tuples from Tg ground dataset. """
    datajson = "tests/tg_ground_data.json"
    with open(datajson) as fp:
        data = json.load(fp)
    abstracts = []
    for k, v in data.items():
        for item in v:
            if item['abstract']:
                abstracts.append((k, item['abstract']))
    return abstracts

bert = BERT()
dataset = Tg_dataset()

      -- ner_ Loaded materials bert.


In [6]:
from backend.text import normalize
from backend import ner_pipeline

In [12]:
for i, (doi, text) in enumerate(dataset):
    text = normalize.normText(text)
    tags = bert.get_tags(text)
    extractor = ner_pipeline.TextDataExtractor(text, tags)
    mat = extractor.extract(debug=True)
    mat._detect_polymer_type()
    # mat._coreference_material_entities()

    if i >= 10:
        break


  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.


/data/akhlak/PromptDataExtraction/_conda_env/lib/python3.10/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
